<a href="https://colab.research.google.com/github/imvktiwari/helllooo/blob/main/murlitrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install Required Libraries
!pip install transformers torch datasets pandas

# Step 2: Import Necessary Libraries
import pandas as pd
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00


In [2]:
# Step 3: Load Your Data
data = pd.read_parquet('/content/senti.parquet')  # Ensure the path is correct
print(data.head())  # Inspect the first few rows of the dataset
print(data.columns)  # Check the column names

# Load the label mapping from the JSON file
with open('/content/labels.json', 'r') as f:
    labels = json.load(f)

print(labels)  # Check the label mapping

                                 ID  \
0  7abbf5aeb011e883c0a47a5e299b371e   
1  1089584daf51e3be29f985bcb935d1fa   
2  1742705348624306b2a6e9c256213808   
3  31aa85f5fd4918a960937825b226b597   
4  37c5947791f4d1a787ba9b1111b0e87b   

                                              Text Polarity    Domain  
0  যাওয়ার সময় যেন স্বাস্থ্যমন্ত্রীকে সাথে নিয়ে যায়       NU  facebook  
1              তার আগে যদি আপনি বিদায় নিতেন স্যার,       NU  facebook  
2   রাষ্ট্রের তহবিল কি একেবারে তলানিতে গিয়ে ঠেকেছে       NU  facebook  
3            সাথে আপনাকে চিপ গেস্ট হিসেবে নিয়ে যাক       NU  facebook  
4     শ্রিপাকে ও রিমান্ডে নিয়ে জিজ্ঞাসাবাদ করা হোক       NU  facebook  
Index(['ID', 'Text', 'Polarity', 'Domain'], dtype='object')
{'SP': 'Strongly Positive', 'WP': 'Weakly Positive', 'NU': 'Neutral', 'WN': 'Weakly Negative', 'SN': 'Strongly Negative'}


In [3]:
# Step 4: Preprocess Data (with padding and truncation)
max_len = 128  # You can adjust this value based on your use case

# Tokenize text with fixed padding and truncation
def tokenize_function(text):
    return tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

# Apply the tokenizer to the 'Text' column
data['inputs'] = data['Text'].apply(lambda x: tokenize_function(x))

# Map sentiment labels to numerical values
label_mapping = {'SP': 0, 'WP': 1, 'NU': 2, 'WN': 3, 'SN': 4}
data['label'] = data['Polarity'].map(label_mapping)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Step 5: Train-test Split
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Step 6: Create Custom Dataset Class (updated)
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, text_data, labels):
        self.input_ids = torch.stack([item['input_ids'].squeeze() for item in text_data])
        self.attention_masks = torch.stack([item['attention_mask'].squeeze() for item in text_data])
        self.labels = torch.tensor(labels.tolist(), dtype=torch.long)  # Ensure labels are tensors

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }

# Create datasets
train_dataset = CustomDataset(train_data['inputs'], train_data['label'])
val_dataset = CustomDataset(val_data['inputs'], val_data['label'])


In [ ]:

# Step 7: Load the MuRIL Sequence Classification Model
model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased", num_labels=5)

# Ensure all model parameters are contiguous
for param in model.parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

# Step 8: Train the Model
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Adjust as necessary
    gradient_accumulation_steps=2,   # To simulate larger batch sizes if needed
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=True,  # Mixed precision training
    logging_dir='./logs',
    evaluation_strategy="epoch",  # Evaluate after every epoch
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Pass the validation set
)

# Start training
trainer.train()


pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
